In [1]:
from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../Data/Multidimensional Knapsack'))
from util import *

from IPython.display import display_html
from itertools import chain, cycle

import time

# Load data

In [2]:
data_folder = '../data/Multidimensional Knapsack/'
# Both Weish and WEING problems are just problems. There is no 
# difference between them, just more instances
# We have Weish[01-30] files in Weish folder
weish_files_no = 30
weish_file_paths = ['weish//Weish'+ str(i).zfill(2) +'.npz' for i in range(1, weish_files_no + 1)]
weish_loaded_files = [np.load(data_folder + i) for i in weish_file_paths]
# We have WEING[1-8] files in Weing folder
weing_files_no = 8
weing_file_paths = ['weing//WEING'+ str(i) +'.npz' for i in range(1, weing_files_no + 1)]
weing_loaded_files = [np.load(data_folder + i) for i in weing_file_paths]
loaded_files = weish_loaded_files + weing_loaded_files
qubo_sizes = [i['n'] for i in loaded_files]
objectives = [i['objective'] for i in loaded_files]
constraints = [i['constraint'] for i in loaded_files]

# Custom methods

In [3]:
def convert_1d_qubo_to_2d(qubo, n):
    if (len(qubo)!= (n) * ((n+1) * 0.5)   + 1):
        print('check that n is the correct size')
        return None, None
    constant = qubo[0]
    linear_terms = np.array(qubo[1:(n + 1)])
    no_of_quadratic_terms = len(qubo) - len(linear_terms) -1
    quadratic_terms = np.array(qubo[-no_of_quadratic_terms:])
    k = 0
    qubo_coeffs = []
    for i in range(n):
        coeffs = []
        for j in range(n):
            if(i == j):
                coeffs.append(linear_terms[i])
            elif(j>i):
                coeffs.append(quadratic_terms[k])
                k+=1
            else:
                coeffs.append(0)
        qubo_coeffs.append(coeffs)
    qubo_coeffs = np.array(qubo_coeffs)

    return qubo_coeffs, constant

In [4]:
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align:left">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [157]:
def new_penalty(qubo_obj, con_obj):
        weights_obj = np.zeros(shape = (len(qubo_obj) * 2), dtype='int64')
        k = 0
        
        for i in range(len(qubo_obj)):
            weights_obj[k]= qubo_obj[i][i]
            weights_obj[k+1]= -qubo_obj[i][i]
            for j in range(len(qubo_obj)):
                if(i!=j):
                    if(qubo_obj[i][j] > 0):
                        weights_obj[k]+= qubo_obj[i][j]
                    else:
                        weights_obj[k+1]-=qubo_obj[i][j]
            k = k+2
            
        weights_con = np.zeros(shape = (len(con_obj) * 2), dtype='int64')
        k = 0
        
        for i in range(len(con_obj)):
            weights_con[k]= con_obj[i][i]
            weights_con[k+1]= -con_obj[i][i]
            for j in range(len(con_obj)):
                if(i!=j):
                    if(con_obj[i][j] > 0):
                        weights_con[k]+= con_obj[i][j]
                    else:
                        weights_con[k+1]-=con_obj[i][j]
            k = k+2
        
        objfunc = max(weights_obj)
        pfunc = min(weights_con[weights_con > 0])
        pfunc = 1 if pfunc < 1 else pfunc
        
        penalty = objfunc//pfunc
        penalty = 1 if penalty < 1 else penalty
    
        return penalty

In [160]:
def minimum_penalty_function(qubo_obj, con_obj):
    weights_con = np.zeros(shape = (len(con_obj) * 2), dtype='int64')
    k = 0
    
    for i in range(len(con_obj)):
        weights_con[k]= con_obj[i][i]
        weights_con[k+1]= -con_obj[i][i]
        for j in range(len(con_obj)):
            if(i!=j):
                if(con_obj[i][j] > 0):
                    weights_con[k]+= con_obj[i][j]
                else:
                    weights_con[k+1]-=con_obj[i][j]
        k = k+2
        # TODO add this everywhere, where we are trying to find a number that is closest to 0
        pfunc = abs(min(weights_con, key=abs))
        pfunc = 1 if pfunc < 1 else pfunc
    
    return pfunc

# Prepare data

In [164]:
obj_qubos, obj_constants, con_qubos, con_constants = [], [], [], []

for i in range(len(qubo_sizes)):
    obj = convert_1d_qubo_to_2d(objectives[i], qubo_sizes[i])
    const = convert_1d_qubo_to_2d(constraints[i], qubo_sizes[i])
    obj_qubos.append(obj[0])
    obj_constants.append(obj[1])
    con_qubos.append(const[0])
    con_constants.append(const[1])

In [168]:
######## QUBO you need to solve ########
# TODO check if verma and lewis penalty is doing everything as expected
vl_penalties = [util.verma_penalty(i) for i in obj_qubos]
new_penalties = [new_penalty(i, j) for (i, j) in zip(obj_qubos, con_qubos)]
min_penalties = [minimum_penalty_function(i, j) for (i, j) in zip(obj_qubos, con_qubos)]
print('V&L penalties', vl_penalties)
print('new penalties', new_penalties)
print(min_penalties)

V&L penalties [892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 892, 30800, 30800, 30800, 30800, 30800, 30800, 107200, 107200]
new penalties [1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 36, 29, 21, 36, 36, 33, 48, 14]
[847, 87, 2495, 547, 2515, 1735, 1695, 1695, 2215, 1095, 1375, 1095, 1095, 1, 835, 1295, 3271, 3031, 3531, 1, 1, 2825, 2791, 2191, 3791, 2191, 2191, 2191, 2191, 791, 847, 1047, 1447, 847, 847, 923, 2191, 7191]


In [169]:
comparison = {'Verma and Lewis' : vl_penalties, 'New' : new_penalties, 'Penalty function minimum' : min_penalties}
pd.DataFrame(comparison)  

,Verma and Lewis,New,Penalty function minimum
0,892,1,847
1,892,10,87
2,892,1,2495
3,892,1,547
4,892,1,2515
5,892,1,1735
6,892,1,1695
7,892,1,1695
8,892,1,2215
9,892,1,1095
